In [4]:
f = open("7.txt", "r")
lines = f.read().splitlines()

In [5]:
from dataclasses import field, dataclass
from typing import TypeVar

FS = TypeVar('FS')

@dataclass
class FS:
    name: str
    parent: FS = None
    dirs: list[FS] = field(default_factory=list)
    files: list[tuple] = field(default_factory=list)

    def goto(self, dname) -> FS:
        if dname == '..':
            return self.parent
        for d in self.dirs:
            if d.name == dname:
                return d

    def total_size(self) -> int:
        s = sum(e[1] for e in self.files)
        s += sum(d.total_size() for d in self.dirs)
        return s

    def add_child(self, name):
        if not any(1 for d in self.dirs if d.name == name):
            self.dirs.append(FS(name, self))

    def add_file(self, name, size):
        if not any(1 for e in self.files if e[1] == name):
            self.files.append((name, size))

In [6]:
print(FS('a'))

FS(name='a', parent=None, dirs=[], files=[])


In [7]:
root = FS('/')
curr = root
readdir = False
for line in lines:
    if line[0] == '$':
        readdir = False
        _, cmd, *n = line.split()
        match cmd:
            case 'cd':
                n,=n
                if n != '/':
                    curr = curr.goto(n)
            case 'ls':
                readdir = True
                continue
    elif readdir:
        a, b, *_ = line.split()
        if a == 'dir':
            curr.add_child(b)
        else:
            curr.add_file(b, int(a))

In [8]:
from pprint import pprint
pprint(root)

FS(name='/',
   parent=None,
   dirs=[FS(name='bjrbjh',
            parent=...,
            dirs=[],
            files=[('ctprm.bpc', 80731)]),
         FS(name='dppgvlh',
            parent=...,
            dirs=[FS(name='rtdnhb',
                     parent=...,
                     dirs=[],
                     files=[('ctprm.bpc', 295880)])],
            files=[('bbjw', 180122), ('ctprm.bpc', 210923), ('hhg', 304465)]),
         FS(name='fcfqp',
            parent=...,
            dirs=[FS(name='cts',
                     parent=...,
                     dirs=[],
                     files=[('nhzpb', 47983)]),
                  FS(name='gjzdf',
                     parent=...,
                     dirs=[FS(name='fcfqp',
                              parent=...,
                              dirs=[],
                              files=[('vcbnl', 145412)])],
                     files=[('vtrhs.mlh', 161310)]),
                  FS(name='hvsnr',
                     parent=...,
     

In [9]:
root.total_size()

43562874

In [10]:
def sum_sizes(n):
    s = n.total_size() if n.total_size() <= 100000 else 0
    return sum(sum_sizes(d) for d in n.dirs) + s
sum_sizes(root)

1886043

In [11]:
print(sum_sizes(root))

1886043


In [12]:
free = 70000000 - root.total_size()
candidate = lambda x: x.total_size() + free >= 30000000

In [21]:
ds = []
def search(n):
    if candidate(n):
        ds.append(n)
    for d in n.dirs:
        search(d)


In [22]:
search(root)
min = ds[0].total_size()
for d in ds:
    if d.total_size() < min:
        min = d.total_size()
print(min)

3842121
